In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import statsmodels.api as sm

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')
sns.set_style('white')
#plt.rcParams["figure.dpi"] = 600
from sklearn.model_selection import train_test_split

In [2]:
fpath = 'Model_Parameters Store model.xlsx'
param_df = pd.read_excel(fpath)



In [3]:
param_df

,parameter,units,description,organism,model,full model,full model per day,MIN model hardcoded parameters,OVERFLOW model hardcoded parameters,EXOENZYME model hardcoded parameters,ROS model hardcoded parameters,MIXOTROPH model hardcoded parameters,Tunable parameters (PRO fitting),Tunable parameters (HET fitting),lower bound,upper bound,logscale fitting
0,Mh,1/sec,HET death rate,HET,All models,1.157407e-06,0.100000,NaN,NaN,NaN,NaN,NaN,No,Yes,1.157407e-07,1.157407e-05,Yes
1,Mp,1/sec,PRO death rate,PRO,All models,1.157407e-06,0.100000,NaN,NaN,NaN,NaN,NaN,Yes,No,2.314815e-07,1.157407e-05,Yes
2,gammaDp,ratio,% of recyclable dead matter of PRO (rest goes ...,PRO,All models,8.000000e-01,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,1.000000e-01,9.000000e-01,Yes
3,gammaDh,ratio,% of recyclable dead matter of HET (rest goes ...,HET,All models,8.000000e-01,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,1.000000e-01,9.000000e-01,Yes
4,Rp,ratio,C:N ratio,PRO,All models,6.625000e+00,NaN,NaN,NaN,NaN,NaN,NaN,No,No,NaN,NaN,NaN
5,Rh,ratio,C:N ratio,HET,All models,5.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,No,No,NaN,NaN,NaN
6,E_leakp,1/sec,organic leakage rate,PRO,OVERFLOW,1.157407e-06,0.100000,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,Yes,No,2.314815e-07,5.787037e-06,Yes
7,E_leakh,1/sec,organic leakage rate,HET,OVERFLOW,1.157407e-06,0.100000,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,No,Yes,1.157407e-07,1.157407e-05,Yes
8,KONp,µmol/L,"organic N K’s (affinity), half-velocity consta...",PRO,MIXOTROPH,1.427508e-01,NaN,1.427508e-01,1.427508e-01,1.427508e-01,1.427508e-01,NaN,Yes,No,2.855016e-02,7.137540e-01,Yes
9,KINp,µmol/L,"inorganic N K’s (affinity), half-velocity cons...",PRO,All models,1.427508e-01,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,2.855016e-02,7.137540e-01,Yes


In [4]:
param_df['param'] = param_df.parameter
param_df['param'] = param_df.param.str.replace('_', '', regex=False)
param_df['param'] = param_df.param.str.replace('gamma', '\\gamma ', regex=False)
param_df['param'] = param_df.param.str.replace('omega', '\\omega ', regex=False)


In [5]:
param_df['param'] = '$' + param_df.param.str.replace(r'[hpHP]$', '_i$', regex=True)

In [6]:
param_df.loc[~param_df.param.str.endswith('$'), 'param'] = param_df.loc[~param_df.param.str.endswith('$'), 'param'] + '$'

In [7]:
param_df.columns

Index(['parameter', 'units', 'description', 'organism', 'model', 'full model',
       'full model per day', 'MIN model hardcoded parameters',
       'OVERFLOW model hardcoded parameters',
       'EXOENZYME model hardcoded parameters',
       'ROS model hardcoded parameters',
       'MIXOTROPH model hardcoded parameters',
       'Tunable parameters (PRO fitting)', 'Tunable parameters (HET fitting)',
       'lower bound', 'upper bound', 'logscale fitting', 'param'],
      dtype='object')

In [8]:
index_col = [
    'model','param', 'units', 'description',  
#    'logscale fitting',
]

value_col = ['full model', 'full model per day', 
       'Tunable parameters (PRO fitting)', 'Tunable parameters (HET fitting)',
       'lower bound', 'upper bound', ]


pdf = param_df.pivot(index=index_col, columns=['organism'], values=value_col)

In [9]:
pdf.dropna(axis=1, how='all', inplace=True)

In [10]:
for i in ('PRO', 'HET'):
    for c in ['full model','full model per day']:
        pdf[(c, i)].fillna(pdf[(c, 'NONE')], inplace=True)    
    

In [11]:
pdf.drop(columns=[            
    ('Tunable parameters (PRO fitting)', 'HET'),
    ('Tunable parameters (PRO fitting)', 'NONE'),
    ('Tunable parameters (HET fitting)', 'PRO'),    
    ('Tunable parameters (HET fitting)', 'NONE'),    
    ('full model','NONE'),
    ('full model per day','NONE'),
        
], inplace=True
        )
pdf.columns

MultiIndex([(                      'full model', 'HET'),
            (                      'full model', 'PRO'),
            (              'full model per day', 'HET'),
            (              'full model per day', 'PRO'),
            ('Tunable parameters (PRO fitting)', 'PRO'),
            ('Tunable parameters (HET fitting)', 'HET'),
            (                     'lower bound', 'HET'),
            (                     'lower bound', 'PRO'),
            (                     'upper bound', 'HET'),
            (                     'upper bound', 'PRO')],
           names=[None, 'organism'])

In [12]:
pdf

full model  \
organism                                                                                                           HET   
model      param                 units                description                                                        
All models $KIC_i$               µmol/L               inorganic C K’s (affinity), half-velocity const...  2.502723e-01   
           $KIN_i$               µmol/L               inorganic N K’s (affinity), half-velocity const...  2.502723e-01   
           $KOC_i$               µmol/L               organic C K’s (affinity), half-velocity constan...  2.502723e-01   
           $KON_i$               µmol/L               organic N K’s (affinity), half-velocity constan...  2.502723e-01   
           $Kmtb_i$              1/sec                Maximum metabolic rate                              3.500000e-05   
           $M_i$                 1/sec                HET death rate                                      1.157407e-06   
                                                      PRO death rate                                               NaN   
           $QCmax_i$             µmol C/L ∙µmol N/L-1 Maximum C/N ratio                                   1.000000e+01   
           $QCmin_i$             µmol C/L ∙µmol N/L-1 Minimum C/N ratio                                   4.000000e+00   
           $QNmax_i$             µmol N/L ∙µmol C/L-1 Maximum nitrogen cell quota                         2.500000e-01   
           $QNmin_i$             µmol N/L ∙µmol C/L-1 Minimum nitrogen cell quota                         1.000000e-01   
           $R_i$                 ratio                C:N ratio                                           5.000000e+00   
           $VmaxIC_i$            1/sec                inorganic C maximum uptake rate                     0.000000e+00   
           $VmaxIN_i$            1/sec                inorganic N maximum uptake rate                     4.812437e-05   
           $VmaxOC_i$            1/sec                organic C maximum uptake rate                       2.165597e-04   
           $VmaxON_i$            1/sec                organic N maximum uptake rate                       4.812437e-05   
           $\gamma DON2DIN$      1/ umolN/L /s        DON2DIN breakdown rate                                       NaN   
           $\gamma D_i$          ratio                % of recyclable dead matter of HET (rest goes t...  8.000000e-01   
                                                      % of recyclable dead matter of PRO (rest goes t...           NaN   
           $b_i$                 no units             respiration coefficient, no units, Geider & Osb...  1.000000e-02   
           $r0_i$                1/sec                dark respiration, sec-1 = 0.18 d-1, Geider & Os...  2.083333e-06   
EXOENZYME  $\gamma DON2DINexo_i$ 1/ umolN/L /s        DON2DIN breakdown rate                              1.157407e-07   
MIXOTROPH  $KOC_i$               µmol/L               organic C K’s (affinity), half-velocity constan...           NaN   
           $KON_i$               µmol/L               organic N K’s (affinity), half-velocity constan...           NaN   
           $VmaxOC_i$            1/sec                organic C maximum uptake rate                                NaN   
           $VmaxON_i$            1/sec                organic N maximum uptake rate                                NaN   
OVERFLOW   $Eleak_i$             1/sec                organic leakage rate                                1.157407e-06   
           $OverflowMode$        bool                 is overlow enabled                                  1.000000e+00   
ROS        $EROS_i$              1/sec                ROS release rate                                    4.050926e-07   
           $KROS_i$              µmol/L               half-velocity constant for ROS breakdown            2.502723e-01   
           $ROSMode$             bool                 is ROS enabled              

In [13]:
pd.set_option('display.float_format', lambda x: '%g' % x)

In [14]:
pdf.rename(columns={
                          'full model' : 'Default',
                  'full model per day' : 'Default day-1',
    'Tunable parameters (PRO fitting)' : 'Tunable',
    'Tunable parameters (HET fitting)' : 'Tunable',
                         'lower bound' : 'Lower bound',
                         'upper bound' : 'Upper bound',
}, level=0, inplace=True)

In [15]:
pdf.columns = [' '.join(c) for c in pdf.columns]

In [16]:
pdf.columns

Index(['Default HET', 'Default PRO', 'Default day-1 HET', 'Default day-1 PRO',
       'Tunable PRO', 'Tunable HET', 'Lower bound HET', 'Lower bound PRO',
       'Upper bound HET', 'Upper bound PRO'],
      dtype='object')

In [17]:
pdf

Default HET  \
model      param                 units                description                                                       
All models $KIC_i$               µmol/L               inorganic C K’s (affinity), half-velocity const...     0.250272   
           $KIN_i$               µmol/L               inorganic N K’s (affinity), half-velocity const...     0.250272   
           $KOC_i$               µmol/L               organic C K’s (affinity), half-velocity constan...     0.250272   
           $KON_i$               µmol/L               organic N K’s (affinity), half-velocity constan...     0.250272   
           $Kmtb_i$              1/sec                Maximum metabolic rate                                  3.5e-05   
           $M_i$                 1/sec                HET death rate                                      1.15741e-06   
                                                      PRO death rate                                              NaN   
           $QCmax_i$             µmol C/L ∙µmol N/L-1 Maximum C/N ratio                                            10   
           $QCmin_i$             µmol C/L ∙µmol N/L-1 Minimum C/N ratio                                             4   
           $QNmax_i$             µmol N/L ∙µmol C/L-1 Maximum nitrogen cell quota                                0.25   
           $QNmin_i$             µmol N/L ∙µmol C/L-1 Minimum nitrogen cell quota                                 0.1   
           $R_i$                 ratio                C:N ratio                                                     5   
           $VmaxIC_i$            1/sec                inorganic C maximum uptake rate                               0   
           $VmaxIN_i$            1/sec                inorganic N maximum uptake rate                     4.81244e-05   
           $VmaxOC_i$            1/sec                organic C maximum uptake rate                        0.00021656   
           $VmaxON_i$            1/sec                organic N maximum uptake rate                       4.81244e-05   
           $\gamma DON2DIN$      1/ umolN/L /s        DON2DIN breakdown rate                                      NaN   
           $\gamma D_i$          ratio                % of recyclable dead matter of HET (rest goes t...          0.8   
                                                      % of recyclable dead matter of PRO (rest goes t...          NaN   
           $b_i$                 no units             respiration coefficient, no units, Geider & Osb...         0.01   
           $r0_i$                1/sec                dark respiration, sec-1 = 0.18 d-1, Geider & Os...  2.08333e-06   
EXOENZYME  $\gamma DON2DINexo_i$ 1/ umolN/L /s        DON2DIN breakdown rate                              1.15741e-07   
MIXOTROPH  $KOC_i$               µmol/L               organic C K’s (affinity), half-velocity constan...          NaN   
           $KON_i$               µmol/L               organic N K’s (affinity), half-velocity constan...          NaN   
           $VmaxOC_i$            1/sec                organic C maximum uptake rate                               NaN   
           $VmaxON_i$            1/sec                organic N maximum uptake rate                               NaN   
OVERFLOW   $Eleak_i$             1/sec                organic leakage rate                                1.15741e-06   
           $OverflowMode$        bool                 is overlow enabled                                            1   
ROS        $EROS_i$              1/sec                ROS release rate                                    4.05093e-07   
           $KROS_i$              µmol/L               half-velocity constant for ROS breakdown               0.250272   
           $ROSMode$             bool                 is ROS enabled                                                1   
           $ROSdecay$            1/sec                ROS decay rate                                      1.15741

In [18]:
pdf.dtypes

Default HET          float64
Default PRO          float64
Default day-1 HET    float64
Default day-1 PRO    float64
Tunable PRO           object
Tunable HET           object
Lower bound HET       object
Lower bound PRO       object
Upper bound HET       object
Upper bound PRO       object
dtype: object

In [19]:
for c in pdf.columns:
    if 'Tunable' not in c:
        print(c)
        pdf[c] = pd.to_numeric(pdf[c])

Default HET
Default PRO
Default day-1 HET
Default day-1 PRO
Lower bound HET
Lower bound PRO
Upper bound HET
Upper bound PRO


In [20]:
pdf.reset_index().to_markdown('params_table.md',index=False  )

In [21]:
pdf.to_excel('params_table_formatted.xlsx')